<a href="https://colab.research.google.com/github/lorenzrossi/InfoRet_SemanticSimilarity/blob/main/InfoRet_Semantic_Similarity_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, classification_report, hamming_loss
import shutil 
import sys
import time, datetime, json
!pip install sentencepiece
!pip install opendatasets
import opendatasets as op

import os

from keras_preprocessing.sequence import pad_sequences

# NLTK
import nltk as nlp
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
nlp.download('stopwords')
nlp.download('popular')
stop_words = stopwords.words('english')

# PYTORCH
!pip install transformers~=2.11.0
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, AdamWeightDecay, get_linear_schedule_with_warmup
import random
import gc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data] 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 674.8/674.8 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 69.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3348e2d67ec5f3fae1e62cfac64446bea80f7177570e7da09af6eb0f337bfc95
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
# downloading te glove package for embedding, for simplicity of computation the "basic" one (6 billion token, 300-dimensional vectors and 400k words) is chosen

!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-01-26 12:50:55--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-01-26 12:50:55--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.96MB/s    in 2m 39s  

2023-01-26 12:53:34 (5.18 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [ ]:
op.download("https://www.kaggle.com/datasets/stanfordu/stanford-natural-language-inference-corpus")
dataset_folder ='stanford-natural-language-inference-corpus'
# bcc30972b8b9f25c2bc6c0a46d8f4d62

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: lorenzrossi
Your Kaggle Key: ··········


100%|██████████| 44.4M/44.4M [00:00<00:00, 94.4MB/s]


In [ ]:
# import the files

train_df = pd.read_csv(os.path.join(dataset_folder, "snli_1.0_train.csv"))
val_df = pd.read_csv(os.path.join(dataset_folder, "snli_1.0_dev.csv"))

In [ ]:
# eliminate any '-' label and null sentence in both train and val test

train_df['sentence1'] = train_df['sentence1'].astype(str)
train_df['sentence2'] = train_df['sentence2'].astype(str)

val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]

train_df = train_df[['gold_label', 'sentence1', 'sentence2']][train_df['gold_label'] != '-']
val_df = val_df[['gold_label', 'sentence1', 'sentence2']][val_df['gold_label'] != '-']

In [ ]:
# subsample train set for computational costs

train_df = train_df.sample(frac=0.3)

In [ ]:
train_df

,gold_label,sentence1,sentence2
65213,neutral,Two black women in fancy uniforms,They are maids.
215793,entailment,A man driving a green jeep is crossing over la...,He moves over uneven ground
488138,entailment,An elderly woman crosses the road in a city.,An elderly woman is in the city.
98726,entailment,"A mother sits against a shabby wall, nursing h...",A mother sits and feeds her baby.
513402,entailment,Two women are talking behind a bar.,Two women are talking at the restaurant.
...,...,...,...
108911,entailment,A woman is walking with her young son.,A young boy and his mother are walking.
517941,contradiction,sailor welcoming passengers onto boat.,The people are boarding an airplane.
345593,neutral,Four snowboarders in different colored outfits...,The snow boards are friends
156295,entailment,The yellow kayak is in the middle of the rapids.,There is a brightly sun-colored contraption in...


In [ ]:
# class to preprocess the datasets, tokenize sentences and obtain the dataloader of both train and val.
# Pairs of sentences 
# The dataloaders provide token ids, mask ids

class SNLIDataset(Dataset):

  def __init__(self, train_df, val_df): # this function takes as input validation and train datasets
    self.label_dict = {'entailment': 1, 'contradiction': 0, 'neutral': 2} # encoding the labels

    self.train_df = train_df
    self.val_df = val_df

    self.base_path = 'stanford-natural-language-inference-corpus' # path reference
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # the tokenizer is the Bert tokenizer
    self.train_data = None
    self.val_data = None
    self.init_data()

  def init_data(self):
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)

  def load_data(self, df):

    MAX_LEN = 512 # setting the maximum number of tokens per sentence equal to 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list() # first sentence
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False) # tokenize the first sentence
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False) # tokenize the second
      # join sentences, separated by a [SEP] special token and between two [CLS] tokens, and retrieve the tokens of each pair of sentences
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids)) # the id representation of each token
      seg_ids.append(segment_ids) # the token type id, which is a binary mask that identifies in which sequence a token belongs.
      mask_ids.append(attention_mask_ids) # a binary mask that identifies whether a token is a real word or just padding. 
                                          # If the token contains [CLS], [SEP], or any real word, then the mask would be 1.
      y.append(self.label_dict[label]) # the actual label

    # padding the maximum length of each sequence. We will use 512, which is the maximum length of a sequence allowed for BERT.
    # the tokens in each sequence that exceed the maximum length will be truncated.  
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

# functions to retrieve the batched tokens 
  def get_data_loaders(self, batch_size):
    train_loader = DataLoader(
      self.train_data,
      shuffle=True,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=False,
      batch_size=batch_size
    )

    return train_loader, val_loader

In [ ]:
dataset = SNLIDataset(train_df, val_df)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

164810
9842


In [ ]:
train_loader, val_loader = dataset.get_data_loaders(batch_size=32)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# select the model

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.to(device)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# setting the optimizer

optimizer = AdamW(model.parameters(), lr=2e-05, eps=1e-08, weight_decay=0.0, correct_bias=True)

In [ ]:
# defining the accuracy

def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [ ]:
EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)

  # Training

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()

      # our model takes the following token inputs and labels and produces the loss and the prediction
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels)

      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)

    # Validation

    model.eval() # turn off train mode
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad(): # we don't want to update the gradient since we want to evaluate the model
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels)
        
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)

    # misc to count how much time it takes
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
train(model, train_loader, val_loader, optimizer)

Epoch 1: train_loss: 0.4529 train_acc: 0.8244 | val_loss: 0.3341 val_acc: 0.8794
00:11:55.76
Epoch 2: train_loss: 0.2974 train_acc: 0.8930 | val_loss: 0.3113 val_acc: 0.8871
00:11:52.97
Epoch 3: train_loss: 0.2133 train_acc: 0.9260 | val_loss: 0.3551 val_acc: 0.8842
00:11:52.95
Epoch 4: train_loss: 0.1541 train_acc: 0.9483 | val_loss: 0.3829 val_acc: 0.8852
00:11:52.95
Epoch 5: train_loss: 0.1195 train_acc: 0.9599 | val_loss: 0.4148 val_acc: 0.8827
00:11:53.17
